In [ ]:
import re
import os
import json
import datetime as dt
import pandas as pd

In [ ]:
def process_date(start_date, end_date):
  """Create helper function to use for starting, ending date of group buy
      Case1: has month day, year in the string
      Case2: Has only month and day -> has to infer from the other dates
      Case3: Sold out
  """
  
  ymd_pattern = '(\D*)\s*(\d{1,2})\s*(?:rd|th|st|nd)*\,*\s*(\d{4})*'
  
  start_tup = re.search(ymd_pattern, start_date)
  start_m, start_d, start_y = start_tup.group(1), start_tup.group(2), start_tup.group(3)
  
  if end_date != 'sold out': 
    end_tup = re.search(ymd_pattern, end_date)
    end_m, end_d, end_y = end_tup.group(1), end_tup.group(2), end_tup.group(3)    
    if end_y is None and start_y is not None: end_y = start_y
  
  if start_y is None and end_y is not None: start_y = end_y  
  
  out_start_date = dt.datetime.strptime(f"{start_d.strip()}-{start_m.strip()}-{start_y.strip()}", '%d-%B-%Y')
  if end_date == 'sold out': out_end_date = end_date
  else:
    out_end_date = dt.datetime.strptime(f"{end_d.strip()}-{end_m.strip()}-{end_y.strip()}", '%d-%B-%Y')
  
  return out_start_date, out_end_date

In [ ]:
scraped_data = json.load(open("scrapped_data.json", "r"))

price_pattern = '(\w*):\s*\*\*\s*\$(\d*\-*\d*)\*\*'
title_pattern = '\[GB\]\s*(.*)\s*//'
date_pattern = '//(.*)-(.*)'

type_list = []
price_list = []
title_list = []
start_list = []
end_list = []

for scrp in scraped_data:
  tup_list = re.findall(price_pattern, scrp['sticky_comment'].replace('\n', ''))
  title = re.findall(title_pattern, scrp['title'].replace('\n', ''))[0] #Title should match only 1 string
  
  # first index is for tuple, second index is for the starting/ending date
  start_date = re.findall(date_pattern, scrp['title'].replace('\n', ''))[0][0].strip().lower()
  end_date = re.findall(date_pattern, scrp['title'].replace('\n', ''))[0][1].strip().lower()
  
  #print(start_date, end_date)
  start_date, end_date = process_date(start_date, end_date)
  
  for tup in tup_list: 
    type_list.append(tup[0])
    price_list.append(tup[1])
    title_list.append(title)
    start_list.append(start_date)
    end_list.append(end_date)

      
df = pd.DataFrame({'title': title_list, 'type': type_list, 'price': price_list, 'start_date': start_list, 'end_date':end_list})
df

In [ ]:
title_list

In [ ]:
price_list

### Distribution of price

### Distribution of Keyboard Type 

### Distribution of the Studio 

### Visualization of number of active group buy in a Year (maybe a stack chart of avaialble group buys/ price need to buy everything, etc.)

### Time from Group Buy to Release (as per the initial announcement data)